In [1]:
from keras.applications import vgg16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Loads the VGG16 model without the top or FC layers
model = vgg16.VGG16(weights='imagenet', include_top = False, input_shape=(img_rows,img_cols,3))

# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

58892288/58889256 [==============================] - 8s 0us/step
0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [2]:
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [3]:
def addTopModel(bottom_model, num_classes, D=256):
    #creates the top or head of the model that will be placed ontop of the bottom layers

    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set the number of classes
num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs = model.input, outputs = FC_Head)

print(modelnew.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [29]:
from keras_preprocessing.image import ImageDataGenerator


path = '/Users/Anil/python codes/coduh2/myself'
train_data_dir =path

validation_data_dir = path

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Set the Batch Size according to your system.
train_batchsize = 10
val_batchsize = 5

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 218 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [30]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("facedetect.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# We put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
modelnew.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 100
nb_validation_samples = 100

# We only train 5 EPOCHS 
epochs = 5
batch_size = 5

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save('facedetect.h5')

Epoch 1/5
10/10 [==============================] - 61s 6s/step - loss: 7.9444 - accuracy: 0.5343 - val_loss: 0.3015 - val_accuracy: 1.0000
Epoch 2/5
10/10 [==============================] - 60s 6s/step - loss: 0.6773 - accuracy: 0.6318 - val_loss: 0.3307 - val_accuracy: 1.0000
Epoch 3/5
10/10 [==============================] - 59s 6s/step - loss: 0.4333 - accuracy: 0.8265 - val_loss: 0.1338 - val_accuracy: 1.0000
Epoch 4/5
10/10 [==============================] - 54s 6s/step - loss: 0.4028 - accuracy: 0.8341 - val_loss: 0.1124 - val_accuracy: 1.0000
Epoch 5/5
10/10 [==============================] - 55s 6s/step - loss: 0.4186 - accuracy: 0.8358 - val_loss: 0.2031 - val_accuracy: 1.0000


In [31]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

m = load_model("facedetect.h5")




In [34]:
#Path to my sample pic
filename = '/Users/Anil/Desktop/trial.jpg'# + '.jpg'

cimage = image.load_img(filename, target_size=(224,224,3))
print(cimage.show())

cimage = image.img_to_array(cimage)
cimage = np.expand_dims(cimage, axis=0)
result = m.predict(cimage)
print(result)
if(result[0][0]>result[0][1]):
    print("Verified")
else:
    print("try again")

None
[[0. 1.]]
try again


In [ ]:
os.remove(path)

In [43]:
import cv2
image = cv2.imread('/Users/Anil/Desktop/trial.jpg')   
cv2.imshow("",image)

In [41]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite('C:/Users/Anil/Desktop/trial2.jpg', image_gray) 

error: OpenCV(4.4.0) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-48c801ya/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
